**Libraries**

In [ ]:
#others
import pandas as pd
import numpy as np

#retrieving data
import requests
import json

#plotting
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import interact, fixed, IntSlider
import matplotlib.pyplot as plt

#optimization
from scipy.optimize import minimize

**Retrieve data**

In [ ]:
#downloading data from coingecko.com
def download_historical_data(crypto_id, days):
    url = f'https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart'

    # Set the number of days for historical data
    params = {
        'vs_currency': 'usd',
        'days': days
    }

    headers = {
        'Content-Type': 'application/json'
    }

    try:
        response = requests.get(url, params=params, headers=headers)
        data = response.json()

        # Save the data to a json file
        filename = f'{crypto_id}_historical_data.json'
        with open(filename, 'w') as f:
            json.dump(data, f)

        print(f"Downloaded historical data for {crypto_id} successfully and saved to {filename}!")

    except requests.exceptions.RequestException as e:
        print(f"Error occurred while downloading data for {crypto_id}: {e}")

# Specify the crypto IDs and the number of days of historical data you want to download
crypto_ids = ['bitcoin', 'ethereum', 'litecoin', 'dogecoin', 'vechain', 'filecoin', 'binancecoin', 'cardano', 'ripple', 'polkadot']
days = 365

# Iterate through the list of crypto IDs and download the data
for crypto_id in crypto_ids:
    download_historical_data(crypto_id, days)
    
dfs = []  # List to store individual DataFrames

for crypto_id in crypto_ids:
    # Load the JSON file
    filename = f'{crypto_id}_historical_data.json'
    with open(filename, 'r') as f:
        data = json.load(f)

    # Extract the price data from the JSON structure
    prices = data['prices']

    # Create a Pandas DataFrame
    df = pd.DataFrame(prices, columns=['Timestamp', f'{crypto_id}_Price'])

    # Convert the timestamp to datetime
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')

    # Round down the timestamp to the nearest day
    df['Timestamp'] = df['Timestamp'].dt.floor('D')

    # Group by timestamp and take the mean of prices
    df = df.groupby('Timestamp').mean()

    # Add the DataFrame to the list
    dfs.append(df)

# Merge all the DataFrames into a single DataFrame based on the Timestamp index
indices = pd.concat(dfs, axis=1)

# Print the merged DataFrame
indices.head()

**Pre-processing**

In [ ]:
indices.isna().sum()
#There is no missing in dataset

In [ ]:
# Handle outliers by winsorizing at 5% and 95% percentiles
pct_low, pct_high = 0.05, 0.95
indices = indices.apply(lambda x: np.clip(x, x.quantile(pct_low), x.quantile(pct_high)))

In [ ]:
#Calculation of the daily and cumulative returns for each asset
returns = indices.pct_change().dropna()
cum_returns = returns.add(1).cumprod().sub(1).dropna()*100

#plotting Cumulative returns
fig = px.line(cum_returns, x=cum_returns.index, y=cum_returns.columns, title='Cumulative Returns of Indices 1Y')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Cumulative Return in %')
fig.show()

In [ ]:
#Calculation rolling volatility for each asset
window_size = 7  # Specify the window size for the rolling calculation
volatility = returns.rolling(window=window_size).std()

#plotting volatility
fig = px.line(volatility, x=cum_returns.index, y=cum_returns.columns, title='Volatility of Indices 1Y')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Volatility')
fig.show()

**Implementing models and evaluation**

In [ ]:
def maximum_sharpe_ratio_portfolio(returns, allow_shorting=True):
    num_assets = returns.shape[1]
    
    # Define the objective function for maximum Sharpe ratio portfolio
    def objective_function(weights):
        # Calculate portfolio mean return
        mean_returns = returns.mean()        
        # Calculate portfolio variance-covariance matrix
        covariance_matrix = returns.cov()
        # Calculate portfolio return and variance
        portfolio_mean = np.sum(mean_returns*weights)
        portfolio_std = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))        
        # Calculate ratio
        sharpe_ratio = portfolio_mean / portfolio_std
        return -sharpe_ratio
    
    # Define the constraints
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1.0})
    
    if allow_shorting:
        bounds = [(None, None)] * num_assets  # No bounds on weights
    else:
        bounds = [(0, None)] * num_assets  # No shorting allowed
    
    # Perform the optimization
    initial_weights = np.ones(num_assets) / num_assets  # Start with equal weights
    optimized_weights = minimize(objective_function, initial_weights, bounds=bounds, constraints=constraints, method='SLSQP').x
    return optimized_weights

def minimum_variance_portfolio(returns, allow_shorting=True):
    num_assets = returns.shape[1]

    def objective_function(weights):
        mean_returns = returns.mean()
        covariance_matrix = returns.cov()
        portfolio_mean = np.sum(mean_returns*weights)
        portfolio_variance = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
        return portfolio_variance
    
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1.0})
    
    if allow_shorting:
        bounds = [(None, None)] * num_assets
    else:
        bounds = [(0, None)] * num_assets
    
    initial_weights = np.ones(num_assets) / num_assets
    optimized_weights = minimize(objective_function, initial_weights, bounds=bounds, constraints=constraints, method='SLSQP').x
    return optimized_weights


# Portfolio Performance Evaluation
def evaluate_portfolio(returns, weights, rebalancing_frequency='daily'):
    n = len(returns)
    if rebalancing_frequency == 'daily':
        returns_rebalanced = returns
    elif rebalancing_frequency == 'weekly':
        returns_rebalanced = returns[::7]
    elif rebalancing_frequency == 'monthly':
        returns_rebalanced = returns[::30]
    else:
        raise ValueError("Invalid rebalancing frequency. Choose from 'daily', 'weekly', or 'monthly'.")
    
    portfolio_returns = np.dot(returns_rebalanced, weights)
    portfolio_cumulative_returns = np.cumprod(portfolio_returns + 1) - 1
    
    portfolio_stats = {
        'Expected Return': np.mean(portfolio_returns),
        'Volatility': np.std(portfolio_returns),
        'Sharpe Ratio': np.mean(portfolio_returns) / np.std(portfolio_returns),
        'Cumulative Returns': portfolio_cumulative_returns[-1]
    }
    
    return portfolio_stats, portfolio_cumulative_returns

In [ ]:
# Compute daily, weekly, and monthly rebalanced portfolios
rebalancing_rules = ['daily', 'weekly', 'monthly']
portfolio_optimization_models = [minimum_variance_portfolio, maximum_sharpe_ratio_portfolio]

for rebalancing_rule in rebalancing_rules:
    for optimization_model in portfolio_optimization_models:
        # Calculate portfolio weights
        weights = optimization_model(returns, allow_shorting=False)
        # Evaluate portfolio performance
        stats, cumulative_returns = evaluate_portfolio(returns, weights, rebalancing_frequency=rebalancing_rule)
        # Calculate asset allocation
        allocation = pd.DataFrame(np.round(weights*100,4),index=returns.columns,columns=['allocation'])
        allocation['allocation'] = allocation['allocation'].map('{:.1f}%'.format)
        # Print portfolio statistics
        print(f"Rebalancing rule: {rebalancing_rule}")
        print(f"Optimization model: {optimization_model.__name__}")
        print(f"Assets:")
        print(allocation.to_string(header=False))
        print(f"Expected returns: {round(stats['Expected Return'],4)}")
        print(f"Cumulative returns: {round(stats['Cumulative Returns'],4)}")
        print(f"Volatility: {round(stats['Volatility'],4)}")
        print(f"Sharpe ratio: {round(stats['Sharpe Ratio'],4)}")
       # Plot cumulative returns
        if rebalancing_rule == 'daily':
            fig = px.line(cumulative_returns, x=cum_returns.index, y=cumulative_returns, title='Cumulative Returns of Indices 1Y')
        elif rebalancing_rule == 'weekly':
            fig = px.line(cumulative_returns, x=cum_returns.index[::7], y=cumulative_returns, title='Cumulative Returns of Indices 1Y')
        elif rebalancing_rule == 'monthly':
            fig = px.line(cumulative_returns, x=cum_returns.index[::30], y=cumulative_returns, title='Cumulative Returns of Indices 1Y')
        else:
            raise ValueError("Invalid rebalancing frequency. Choose from 'daily', 'weekly', or 'monthly'.")
        fig.update_xaxes(title_text='Date')
        fig.update_yaxes(title_text='Cumulative Return in %')
        fig.show()
        print("-"*80)

In [ ]:
def portfolio_performance(weights, mean_returns, cov_matrix):
    """
    Calculates the portfolio's standard deviation and annualized return.
    
    Args:
        weights (np.ndarray): Array of portfolio weights.
        mean_returns (pd.Series): Series of mean returns for each asset.
        cov_matrix (pd.DataFrame): Covariance matrix of asset returns.
    
    Returns:
        Tuple: Standard deviation and annualized return of the portfolio.
    """
    returns = np.sum(mean_returns*weights)*365
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))*np.sqrt(365)
    return std, returns

def random_portfolios(num_portfolios, mean_returns, cov_matrix):
    """
    Generates random portfolios with given number of portfolios and asset information.
    
    Args:
        num_portfolios (int): Number of random portfolios to generate.
        mean_returns (pd.Series): Series of mean returns for each asset.
        cov_matrix (pd.DataFrame): Covariance matrix of asset returns.
    
    Returns:
        Tuple: Array of portfolio standard deviations, returns, and Sharpe ratios, and a list of portfolio weights.
    """
    results = np.zeros((3,num_portfolios))
    weights_record = []
    
    num_assets = len(mean_returns)
    
    for i in range(num_portfolios):
        weights = np.random.random(num_assets)
        weights /= np.sum(weights)
        weights_record.append(weights)
        portfolio_std_dev, portfolio_return = portfolio_performance(weights, mean_returns, cov_matrix)
        results[0,i] = portfolio_std_dev
        results[1,i] = portfolio_return
        results[2,i] = portfolio_return / portfolio_std_dev
    
    return results, weights_record

def efficient_return(mean_returns, cov_matrix, target):
    """
    Finds the weights for an efficient portfolio given a target return.
    
    Args:
        mean_returns (pd.Series): Series of mean returns for each asset.
        cov_matrix (pd.DataFrame): Covariance matrix of asset returns.
        target (float): Target portfolio return.
    
    Returns:
        scipy.optimize.OptimizeResult: Result of the optimization with the weights for the efficient portfolio.
    """
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)

    def portfolio_return(weights):
        return portfolio_performance(weights, mean_returns, cov_matrix)[1]
    
    def portfolio_volatility(weights, mean_returns, cov_matrix):
        return portfolio_performance(weights, mean_returns, cov_matrix)[0]

    constraints = ({'type': 'eq', 'fun': lambda x: portfolio_return(x) - target},
                   {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0,1) for asset in range(num_assets))
    result = minimize(portfolio_volatility, num_assets*[1./num_assets,], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def efficient_frontier(mean_returns, cov_matrix, returns_range):
    """
    Generates a list of efficient portfolios along the efficient frontier.
    
    Args:
        mean_returns (pd.Series): Series of mean returns for each asset.
        cov_matrix (pd.DataFrame): Covariance matrix of asset returns.
        returns_range (np.ndarray): Array of target returns.
    
    Returns:
        List: List of OptimizeResult objects representing the efficient portfolios.
    """
    efficients = []
    for ret in returns_range:
        efficients.append(efficient_return(mean_returns, cov_matrix, ret))
    return efficients

In [ ]:
def display_calculated_ef_with_random(returns, portfolio_optimization_models, num_portfolios=10000, interactive=False):
    num_assets = returns.shape[1]
    # Calculate portfolio mean return
    mean_returns = returns.mean()
    # Calculate portfolio variance-covariance matrix
    covariance_matrix = returns.cov()
    def plot_efficient_frontier(num_portfolios):
        # Generate random portfolios
        results, _ = random_portfolios(num_portfolios, mean_returns, covariance_matrix)
        results=pd.DataFrame(results).T
        results = results.rename(columns={2: 'Sharpe Ratio'})
        # Plot scatter plot of random portfolios
        fig = px.scatter(results, x=0, y=1, color='Sharpe Ratio', color_continuous_scale='YlGnBu', opacity=0.3,
                         labels={0: 'Volatility', 1: 'Returns'})
        # Plot optimized portfolios
        for optimization_model in portfolio_optimization_models:
            weights = optimization_model(returns, allow_shorting=False)
            sdp, rp = portfolio_performance(weights, mean_returns, covariance_matrix)
            fig.add_trace(go.Scatter(x=[sdp], y=[rp], mode='markers', name=optimization_model.__name__))
        # Plot efficient frontier
        target = np.linspace(0, 0.65, 50)
        efficient_portfolios = efficient_frontier(mean_returns, covariance_matrix, target)
        fig.add_trace(go.Scatter(x=[p['fun'] for p in efficient_portfolios], y=target,
                                 mode='lines', line=dict(dash='dash'), name='Efficient Frontier'))
        # Update layout and display the plot
        fig.update_layout(title='Calculated Portfolio Optimization based on Efficient Frontier',
                          xaxis_title='Volatility (annualized)', yaxis_title='Returns (annualized)', showlegend=True)
        fig.update_layout(legend=dict(x=0, y=1, traceorder='normal', font=dict(size=12), bgcolor='LightSteelBlue', bordercolor='Black', borderwidth=2))
        fig.show()
    # Create interactive widget
    if interactive:
        interact(plot_efficient_frontier, num_portfolios=IntSlider(min=10000, max=100000, step=10000, continuous_update=False))
    else:
        return plot_efficient_frontier(num_portfolios)

In [ ]:
display_calculated_ef_with_random(returns, portfolio_optimization_models, num_portfolios=100000)

In [ ]:
display_calculated_ef_with_random(returns, portfolio_optimization_models,  num_portfolios=10000, interactive=True)

**Q-learning algorithm for automated stock trading - additional part**

In [ ]:
from collections import deque
import random
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()

In [ ]:
class Agent:
    def __init__(self, state_size, window_size, trend, skip, batch_size):
        # Initialize the Agent object with necessary attributes
        self.state_size = state_size
        self.window_size = window_size
        self.half_window = window_size // 2
        self.trend = trend
        self.skip = skip
        self.action_size = 3
        self.batch_size = batch_size
        self.memory = deque(maxlen=1000)
        self.inventory = []
        self.gamma = 0.95
        self.epsilon = 0.5
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.999
        
        # TensorFlow graph setup
        tf.reset_default_graph()
        self.sess = tf.InteractiveSession()
        self.X = tf.placeholder(tf.float32, [None, self.state_size])
        self.Y = tf.placeholder(tf.float32, [None, self.action_size])
        feed = tf.layers.dense(self.X, 256, activation=tf.nn.relu)
        self.logits = tf.layers.dense(feed, self.action_size)
        self.cost = tf.reduce_mean(tf.square(self.Y - self.logits))
        self.optimizer = tf.train.GradientDescentOptimizer(1e-5).minimize(self.cost)
        self.sess.run(tf.global_variables_initializer())
        
    def act(self, state):
        # Choose an action based on the current state
        if random.random() <= self.epsilon:
            return random.randrange(self.action_size)
        return np.argmax(self.sess.run(self.logits, feed_dict={self.X: state})[0])
    
    def get_state(self, t):
        # Generate the state representation for a given time step
        window_size = self.window_size + 1
        d = t - window_size + 1
        block = self.trend[d : t + 1] if d >= 0 else -d * [self.trend[0]] + self.trend[0 : t + 1]
        res = []
        for i in range(window_size - 1):
            res.append(block[i + 1] - block[i])
        return np.array([res])
    
    def replay(self, batch_size):
        # Replay memory to train the agent
        mini_batch = []
        l = len(self.memory)
        for i in range(l - batch_size, l):
            mini_batch.append(self.memory[i])
        replay_size = len(mini_batch)
        X = np.empty((replay_size, self.state_size))
        Y = np.empty((replay_size, self.action_size))
        states = np.array([a[0][0] for a in mini_batch])
        new_states = np.array([a[3][0] for a in mini_batch])
        Q = self.sess.run(self.logits, feed_dict={self.X: states})
        Q_new = self.sess.run(self.logits, feed_dict={self.X: new_states})
        for i in range(len(mini_batch)):
            state, action, reward, next_state, done = mini_batch[i]
            target = Q[i]
            target[action] = reward
            if not done:
                target[action] += self.gamma * np.amax(Q_new[i])
            X[i] = state
            Y[i] = target
        cost, _ = self.sess.run([self.cost, self.optimizer], feed_dict={self.X: X, self.Y: Y})
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        return cost
    
    def buy(self, initial_money):
        # Buy/sell actions based on the current state and available money
        starting_money = initial_money
        states_sell = []
        states_buy = []
        inventory = []
        state = self.get_state(0)
        for t in range(0, len(self.trend) - 1, self.skip):
            action = self.act(state)
            next_state = self.get_state(t + 1)
            if action == 1 and initial_money >= self.trend[t] and t < (len(self.trend) - self.half_window):
                inventory.append(self.trend[t])
                initial_money -= self.trend[t]
                states_buy.append(t)
                print('day %d: buy 1 unit at price %f, total balance %f' % (t, self.trend[t], initial_money))
            elif action == 2 and len(inventory):
                bought_price = inventory.pop(0)
                initial_money += self.trend[t]
                states_sell.append(t)
                try:
                    invest = ((self.trend[t] - bought_price) / bought_price) * 100
                except:
                    invest = 0
                print(
                    'day %d, sell 1 unit at price %f, total balance %f'
                    % (t, self.trend[t], initial_money)
                )
            state = next_state
        invest = ((initial_money - starting_money) / starting_money) * 100
        total_gains = initial_money - starting_money
        return states_buy, states_sell, total_gains, invest
    
    def train(self, iterations, checkpoint, initial_money):
        # Train the agent for a certain number of iterations
        for i in range(iterations):
            total_profit = 0
            inventory = []
            state = self.get_state(0)
            starting_money = initial_money
            for t in range(0, len(self.trend) - 1, self.skip):
                action = self.act(state)
                next_state = self.get_state(t + 1)
                if action == 1 and starting_money >= self.trend[t] and t < (len(self.trend) - self.half_window):
                    inventory.append(self.trend[t])
                    starting_money -= self.trend[t]
                elif action == 2 and len(inventory) > 0:
                    bought_price = inventory.pop(0)
                    total_profit += self.trend[t] - bought_price
                    starting_money += self.trend[t]
                invest = ((starting_money - initial_money) / initial_money)
                self.memory.append(
                    (state, action, invest, next_state, starting_money < initial_money)
                )
                state = next_state
                batch_size = min(self.batch_size, len(self.memory))
                cost = self.replay(batch_size)
            if (i + 1) % checkpoint == 0:
                print(
                    'epoch: %d, reward: %f.3, loss: %f'
                    % (i + 1, total_profit, cost)
                )

In [ ]:
chosen_crypto = ['litecoin_Price', 'vechain_Price', 'filecoin_Price']

#initialization
initial_money = 100000
window_size = 30
skip = 1
batch_size = 32

for crypto_id in chosen_crypto:
    #df of crypto chosen
    df = indices[crypto_id].values.tolist()
    #defining agent
    agent = Agent(state_size = window_size, 
              window_size = window_size, 
              trend = df, 
              skip = skip, 
              batch_size = batch_size)
    #training
    agent.train(iterations = 100, checkpoint = 10, initial_money = initial_money)
    states_buy, states_sell, total_gains, invest = agent.buy(initial_money = initial_money)
    #plotting results
    fig = plt.figure(figsize = (15,5))
    plt.plot(df, color='r', lw=2.)
    plt.plot(df, '^', markersize=10, color='m', label = 'buying signal', markevery = states_buy)
    plt.plot(df, 'v', markersize=10, color='k', label = 'selling signal', markevery = states_sell)
    plt.title(f'Buy/sell for {crypto_id} - at the end number of coins: %d, value of coins: %.2f, balance: %.2f, total gain: %.2f' % (len(states_buy) - len(states_sell), round((len(states_buy) - len(states_sell)) * indices['litecoin_Price'][-1], 2), initial_money + total_gains, round((len(states_buy) - len(states_sell)) * indices['litecoin_Price'][-1] + total_gains, 2)))
    plt.legend()
    plt.show()